In [5]:
conda install -c anaconda pandasql

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\asaif\anaconda3

  added / updated specs:
    - pandasql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.01.10 |       haa95532_0         158 KB  anaconda
    certifi-2022.12.7          |  py310haa95532_0         153 KB  anaconda
    conda-23.1.0               |  py310haa95532_0         990 KB  anaconda
    openssl-1.1.1u             |       h2bbff1b_0         5.5 MB
    pandasql-0.7.3             |  py310haa95532_1          32 KB  anaconda
    ------------------------------------------------------------
                                           Total:         6.8 MB

The following NEW packages will be INSTALLED:

  pandasql           anaconda/win-64::pandasql-0.7.3-py310haa95532_1 

The following packages will be UPDATED:

  openssl                                 1.1.1t



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0




In [7]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import math
import os
import sys

In [60]:
# Participant to process
# 処理する参加者
USER = 'participant1'

# Collection dataset
# コレクションデータセット
# COLLECTION = '2-person'
COLLECTION = '10-person'

# Define train data path
# 訓練データのパスを定義
TRAIN_DATA_PATH = f'Train Dataset/Train Dataset/dataset/{COLLECTION}'

# Define test data path
# テストデータのパスを定義
TEST_DATA_PATH = 'TestDataset\\'

# Desired interval 望ましい間隔
#   * 15min: 15 minutes
#   * 30min: 30 minutes
#   * 1h: 1 hour
#   * 15s: 15 seconds
TARGET_FREQ_AS_INT = 15     # 15|1
TARGET_FREQ_UNIT1 = 'min'   # min|s
TARGET_FREQ_UNIT2 = 'm'     # m|s
TARGET_FREQ = f'{TARGET_FREQ_AS_INT}{TARGET_FREQ_UNIT1}'

# Wearing-off columns for renaming where value is the output
# 出力値の場合のリネーム用のウェアリングオフカラム
WEARING_OFF_COLUMNS = {
  "Timestamp": "timestamp",
  "Wearing Off": "wearing_off",
  "started_at": "wo_start", "finished_at": "wo_end",
  "tremors": "wo_tremors",
  "slowdown": "wo_slowdown",
  "moodchange": "wo_moodchange",
  "rigidity": "wo_rigidity",
  "pain": "wo_pain",
  "impairment_hands": "wo_impairment_hands",
  "slow_thoughts": "wo_slow_thoughts",
  "anxiety": "wo_anxiety",
  "muscle_spasm": "wo_muscle_spasm"
}

# Drug intake columns for renaming where value is the output
# 出力値の場合のリネーム用の薬の摂取カラム
DRUG_INTAKE_COLUMNS = {
  "started_at": "drug_intake_start",
  "finished_at": "drug_intake_end",
  "tremors": "drug_intake_tremors",
  "slowdown": "drug_intake_slowdown",
  "moodchange": "drug_intake_moodchange",
  "rigidity": "drug_intake_rigidity",
  "pain": "drug_intake_pain",
  "impairment_hands": "drug_intake_impairment_hands",
  "slow_thoughts": "drug_intake_slow_thoughts",
  "anxiety": "drug_intake_anxiety",
  "muscle_spasm": "drug_intake_muscle_spasm"
}

# Define test periods
# テスト期間を定義
test_horizons = {
  "participant1": ["2021-12-02 0:00", "2021-12-03 23:45"],
  "participant2": ["2021-11-28 0:00", "2021-11-29 23:45"],
  "participant3": ["2021-11-25 0:00", "2021-11-26 23:45"],
  "participant4": ["2021-12-06 0:00", "2021-12-07 7:15"],
  "participant5": ["2021-11-28 0:00", "2021-11-29 23:45"],
  "participant6": ["2021-12-06 0:00", "2021-12-07 23:45"],
  "participant7": ["2021-12-12 0:00", "2021-12-13 9:45"],
  "participant8": ["2021-12-23 0:00", "2021-12-24 23:45"],
  "participant9": ["2021-12-23 0:00", "2021-12-24 23:45"],
  "participant10": ["2021-12-23 0:00", "2021-12-24 23:45"],
}

# Convert test periods to DataFrame
# テスト期間をDataFrameに変換
test_horizons_df = pd.DataFrame(
  [[participant, test_start_date, test_end_date]
   for participant, (test_start_date, test_end_date) in test_horizons.items()],
  columns=['participant', 'test_start_date', 'test_end_date']
)

In [61]:
heart_rate = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\garmin.xlsx',
                           sheet_name='heart_rate', index_col='Timestamp'
                           ).query(f'participant == "{USER}"')


In [62]:
heart_rate_test = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Test Dataset\\Test Dataset\\garmin.xlsx',
                                sheet_name='heart_rate', index_col='Timestamp'
                                ).query(f'participant == "{USER}"')

In [63]:
heart_rate.head(10)

,Unnamed: 0,heart_rate,participant
Timestamp,,,
2021-11-25 00:37:15,0,62,participant1
2021-11-25 00:37:30,1,62,participant1
2021-11-25 00:37:45,2,62,participant1
2021-11-25 00:38:00,3,62,participant1
2021-11-25 00:41:15,4,74,participant1
2021-11-25 00:41:30,5,74,participant1
2021-11-25 00:41:45,6,74,participant1
2021-11-25 00:42:00,7,74,participant1
2021-11-25 00:42:15,8,74,participant1


In [64]:

steps = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\garmin.xlsx',
                      sheet_name='steps', index_col='Timestamp'
                      ).query(f'participant == "{USER}"')

steps_test = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Test Dataset\\Test Dataset\\garmin.xlsx',
                           sheet_name='steps', index_col='Timestamp'
                           ).query(f'participant == "{USER}"')

In [65]:
stress = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\garmin.xlsx',
                       sheet_name='stress', index_col='Timestamp'
                       ).query(f'participant == "{USER}"')

stress_test = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Test Dataset\\Test Dataset\\garmin.xlsx',
                            sheet_name='stress', index_col='Timestamp'
                            ).query(f'participant == "{USER}"')

In [66]:
sleep = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\garmin.xlsx',
                      sheet_name='sleep', index_col='Calendar Date'
                      ).query(f'participant == "{USER}"')

sleep_test = pd.read_excel(f'D:\\UEL\\Disertation\\Parkinsons Disease\\Test Dataset\\Test Dataset\\garmin.xlsx',
                           sheet_name='sleep', index_col='Calendar Date'
                           ).query(f'participant == "{USER}"')

In [67]:
wearing_off_periods = pd.read_excel(
  f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\wearing_off_periods.xlsx',
  sheet_name='wearing_off_periods', index_col='wearing_off_id'
).query(
  f'participant == "{USER}"'
).drop(columns=['participant'])

In [68]:
drug_intake_with_symptoms = pd.read_excel(
  f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Train Dataset\\dataset\\drug_intake_with_symptoms.xlsx',
  sheet_name='drug_intake_with_symptoms', index_col='drug_intake_id'
).query(
  f'participant == "{USER}"'
).drop(columns=['participant'])

In [69]:
def complete_reference(df, target_column, set, freq='15s'):
  # Ensures the max date of train set is last available before test set
  # トレインセットの最大日付がテストセットの前に利用可能な最後の日付であることを確認します
  train_max_date = pd.to_datetime(
    test_horizons_df.query(
      f'participant == "{USER}"'
    ).test_start_date,
    format='%Y-%m-%d %H:%M'
  ) - pd.Timedelta(freq)

  # Ensures the min date of test set is the start of test set
  # テストセットの最小日付がテストセットの開始日であることを確認します
  test_min_date = pd.to_datetime(
    test_horizons_df.query(
      f'participant == "{USER}"'
    ).test_start_date,
    format='%Y-%m-%d %H:%M'
  )

  # Create reference dataframe based on the start date and end date.
  # 開始日と終了日に基づいて参照データフレームを作成します。
  if set == 'train':
    reference = pd.DataFrame(
      index=pd.date_range(
        start=df.index.min(),
        end=train_max_date.max(),
        freq=freq,
        name='Timestamp'
      )
    )
  elif set == 'test':
    reference = pd.DataFrame(
      index=pd.date_range(
        start=test_min_date.min(),
        end=df.index.max(),
        freq=freq,
        name='Timestamp'
      )
    )
  else:
    # Base case to get the min and max from the data
    # データから最小値と最大値を取得するためのベースケース
    reference = pd.DataFrame(
      index=pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq=freq,
        name='Timestamp'
      )
    )

  # Resample by dataset's interval
  #   Merge reference with heart rate data
  #   Fill missing values based on dataset's handling of missing values
  # データセットの間隔でリサンプリングします
  #   参照と心拍数データをマージします
  #   欠損値の処理に基づいて欠損値を埋めます
  df = reference.merge(
    df.resample(freq).agg({
      target_column: 'mean',
      'participant': 'first'
    }), on='Timestamp', how='left'
  ).fillna({
    target_column: -1,
    'participant': USER
  })
  return df


# Garmin's heart rate interval
# ガーミンの心拍数間隔
heart_rate_freq = '15s'
heart_rate = complete_reference(heart_rate,
                                target_column='heart_rate',
                                set='train',
                                freq=heart_rate_freq)
heart_rate_test = complete_reference(heart_rate_test,
                                     target_column='heart_rate',
                                     set='test',
                                     freq=heart_rate_freq)

# Garmin's step interval
# ガーミンのステップ間隔
steps_freq = '15min'
steps = complete_reference(steps,
                           target_column='steps',
                           set='train',
                           freq=steps_freq)
steps_test = complete_reference(steps_test,
                                target_column='steps',
                                set='test',
                                freq=steps_freq)

# Garmin's stress interval
# ガーミンのストレス間隔
stress_freq = '3min'
stress = complete_reference(stress,
                            target_column='stress_score',
                            set='train',
                            freq=stress_freq)
stress_test = complete_reference(stress_test,
                                 target_column='stress_score',
                                 set='test',
                                 freq=stress_freq)

In [70]:
def transform_sleep_duration_to_sleep_per_day(df):
  # Compute duration in minutes
  # 分単位で期間を計算します
  df['duration'] = (df['End Time'] - df['Start Time']) / np.timedelta64(1, 'm')

  # Transform sleep data by sleep stage
  # 睡眠段階ごとに睡眠データを変換します
  df = df.pivot_table(
    index='Calendar Date',
    columns='Sleep Type',
    values='duration',
    aggfunc='sum'
  )

  # Fill missing sleep stage with 0 minutes
  # 欠損している睡眠段階を0分で埋めます
  df = pd.DataFrame(df.to_records(),
                    columns=['Calendar Date',
                             'awake', 'light',
                             'rem', 'deep']
                    ).set_index('Calendar Date').fillna(0)

  # Set index as DateTimeIndex type
  # インデックスをDateTimeIndex型に設定します
  df.index = pd.to_datetime(df.index)
  df.index.name = 'Timestamp'

  # Compute other sleep features
  # その他の睡眠機能を計算します
  df['nonrem_total'] = (df['deep'] + df['light'])
  df['total'] = (df['nonrem_total'] + df['rem'])
  df['nonrem_percentage'] = df['nonrem_total'] / df['total']
  df['sleep_efficiency'] = df['total'] / (df['total'] + df['awake'])

  # Ignore unmeasurable sleep classifcation
  # 測定不能な睡眠分類を無視します
  if 'unmeasurable' in df.columns:
    df.drop(columns='unmeasurable', inplace=True)

  return df


sleep = transform_sleep_duration_to_sleep_per_day(sleep)
sleep_test = transform_sleep_duration_to_sleep_per_day(sleep_test)

In [71]:

# Train Set
# 訓練セット
start = [heart_rate.index.min(),
         steps.index.min(),
         stress.index.min(),
         sleep.index.min()]

end = [heart_rate.index.max(),
       steps.index.max(),
       stress.index.max(),
       sleep.index.max()]

# Remove NaT values from the start and end lists
start = min(date for date in start if not pd.isnull(date))
end = max(date for date in end if not pd.isnull(date))

# Check if heart_rate index has NaT
heart_rate_min = heart_rate.index.min()
heart_rate_max = heart_rate.index.max()

if pd.isnull(heart_rate_min):
    heart_rate_min = "No valid time"
if pd.isnull(heart_rate_max):
    heart_rate_max = "No valid time"

print("Check min and max value from train set")
print(f"min date: {start}")
print(
    f'Heart rate: {heart_rate_min}\n',
    f'Steps: {steps.index.min()}\n',
    f'Stress: {stress.index.min()}\n',
    f'Sleep: {sleep.index.min()}\n',
)

print(f"max date: {end}")
print(
    f'Heart rate: {heart_rate_max}\n',
    f'Steps: {steps.index.max()}\n',
    f'Stress: {stress.index.max()}\n',
    f'Sleep: {sleep.index.max()}\n',
)

Check min and max value from train set
min date: 2021-11-25 00:00:00
Heart rate: 2021-11-25 00:37:15
 Steps: 2021-11-25 00:00:00
 Stress: 2021-11-25 00:00:00
 Sleep: 2021-11-26 00:00:00

max date: 2021-12-01 23:59:45
Heart rate: 2021-12-01 23:59:45
 Steps: 2021-12-01 23:45:00
 Stress: 2021-12-01 23:57:00
 Sleep: 2021-12-01 00:00:00



In [72]:
# Test Set
# テストセット
start_test = [heart_rate_test.index.min(),
              steps_test.index.min(),
              stress_test.index.min(),
              sleep_test.index.min()]

end_test = [heart_rate_test.index.max(),
            steps_test.index.max(),
            stress_test.index.max(),
            sleep_test.index.max()]

# Get minimum and maximum date from start
# 開始日から最小日と最大日を取得します
start_test = min(start_test)
end_test = max(end_test)

print("Check min and max value from test set")
print(f"min date: {start_test}")
print(
  f'Heart rate: {heart_rate_test.index.min()}\n',
  f'Steps: {steps_test.index.min()}\n',
  f'Stress: {stress_test.index.min()}\n',
  f'Sleep: {sleep_test.index.min()}\n',
)

print(f"max date: {end_test}")
print(
  f'Heart rate: {heart_rate_test.index.max()}\n',
  f'Steps: {steps_test.index.max()}\n',
  f'Stress: {stress_test.index.max()}\n',
  f'Sleep: {sleep_test.index.max()}\n',
)

Check min and max value from test set
min date: 2021-12-02 00:00:00
Heart rate: 2021-12-02 00:00:00
 Steps: 2021-12-02 00:00:00
 Stress: 2021-12-02 00:00:00
 Sleep: 2021-12-02 00:00:00

max date: 2021-12-03 23:59:45
Heart rate: 2021-12-03 23:59:45
 Steps: 2021-12-03 23:45:00
 Stress: 2021-12-03 23:57:00
 Sleep: 2021-12-03 00:00:00



In [73]:
# Create reference dataframe based on train set's start date and end date.
# 訓練セットの開始日と終了日に基づいて参照データフレームを作成します。
reference = pd.DataFrame(
  index=pd.date_range(
    start=start,
    end=end,
    freq=TARGET_FREQ,
    name='Timestamp'
  )
)

# Combine each Garmin dataset to reference
#   * mean: get the average
#   * first: get the first value i.e., the value on that time
#   * last: get the last value i.e., the value on the previous time
# For example,
#   .resample(target_freq).agg({
#     'heart_rate': 'mean',
#   })
# 各Garminデータセットを結合して参照する
# 平均値を取得する
# * first: 最初の値、つまりその時刻の値を取得する。
# * last: 最後の値、つまり前の時刻の値を得る
# 例えば
# .resample(target_freq).agg({)
# 'heart_rate': 'mean'、
# })
garmin_data = reference.merge(
  # Resample heart rate (15sec) to target frequency (e.g., 15min)
  # 心拍数（15秒）をターゲット周波数（例：15分）にリサンプリングします
  heart_rate.resample(TARGET_FREQ).agg({
    'heart_rate': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample steps (15min) to target frequency (e.g., 15min)
  # ステップ（15分）をターゲット周波数（例：15分）にリサンプリングします
  steps.resample(TARGET_FREQ).agg({
    'steps': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample stress (3min) to target frequency (e.g., 15min)
  # ストレス（3分）をターゲット周波数（例：15分）にリサンプリングします
  stress.resample(TARGET_FREQ).agg({
    'stress_score': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample sleep (1d) to target frequency (e.g., 15min)
  # 睡眠（1日）をターゲット周波数（例：15分）にリサンプリングします
  sleep.resample(TARGET_FREQ).mean(), on='Timestamp', how='left'
).ffill()

print(garmin_data.head(7).index)
print(garmin_data.tail(7).index)

DatetimeIndex(['2021-11-25 00:00:00', '2021-11-25 00:15:00',
               '2021-11-25 00:30:00', '2021-11-25 00:45:00',
               '2021-11-25 01:00:00', '2021-11-25 01:15:00',
               '2021-11-25 01:30:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)
DatetimeIndex(['2021-12-01 22:15:00', '2021-12-01 22:30:00',
               '2021-12-01 22:45:00', '2021-12-01 23:00:00',
               '2021-12-01 23:15:00', '2021-12-01 23:30:00',
               '2021-12-01 23:45:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)


In [74]:
# Create reference dataframe based on test set's start date and end date.
# テストセットの開始日と終了日に基づいて参照データフレームを作成します。
reference = pd.DataFrame(
  index=pd.date_range(
    start=start_test,
    end=end_test,
    freq=TARGET_FREQ,
    name='Timestamp'
  )
)

# Combine each Garmin dataset to reference
#   * mean: get the average
#   * first: get the first value i.e., the value on that time
#   * last: get the last value i.e., the value on the previous time
# For example,
#   .resample(target_freq).agg({
#     'heart_rate': 'mean',
#   })
# 各Garminデータセットを結合して参照する
# 平均値を取得する
# * first: 最初の値、つまりその時刻の値を取得する。
# * last: 最後の値、つまり前の時刻の値を得る
# 例えば
# .resample(target_freq).agg({)
# 'heart_rate': 'mean'、
# })
garmin_data_test = reference.merge(
  # Resample heart rate (15sec) to target frequency (e.g., 15min)
  # 心拍数（15秒）をターゲット周波数（例：15分）にリサンプリングします
  heart_rate_test.resample(TARGET_FREQ).agg({
    'heart_rate': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample steps (15min) to target frequency (e.g., 15min)
  # ステップ（15分）をターゲット周波数（例：15分）にリサンプリングします
  steps_test.resample(TARGET_FREQ).agg({
    'steps': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample stress (3min) to target frequency (e.g., 15min)
  # ストレス（3分）をターゲット周波数（例：15分）にリサンプリングします
  stress_test.resample(TARGET_FREQ).agg({
    'stress_score': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample sleep (1d) to target frequency (e.g., 15min)
  # 睡眠（1日）をターゲット周波数（例：15分）にリサンプリングします
  sleep_test.resample(TARGET_FREQ).mean(), on='Timestamp', how='left'
).ffill()

print(garmin_data_test.head().index)
print(garmin_data_test.tail().index)

DatetimeIndex(['2021-12-02 00:00:00', '2021-12-02 00:15:00',
               '2021-12-02 00:30:00', '2021-12-02 00:45:00',
               '2021-12-02 01:00:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)
DatetimeIndex(['2021-12-03 22:45:00', '2021-12-03 23:00:00',
               '2021-12-03 23:15:00', '2021-12-03 23:30:00',
               '2021-12-03 23:45:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)


In [75]:
# Train set
# 訓練セット

# Match wearing-off periods to garmin timestamp by left join
#   Left join when wearing-off period start and end time is between garmin timestamp
# 左結合により、着用オフ期間をガーミンタイムスタンプに一致させます
#   着用オフ期間の開始時刻と終了時刻がガーミンタイムスタンプの間にある場合は、左結合します
def pysqldf(q): return sqldf(q, globals())


cond_join = '''
  select 
    garmin.*,
    wearing_off_periods.*,
    case
      when wearing_off_periods.[started_at] is not null THEN 1
    else 0
    end as 'Wearing Off'
  from garmin_data as garmin
  left join wearing_off_periods
  on garmin.[Timestamp] BETWEEN
    wearing_off_periods.[started_at] AND wearing_off_periods.[finished_at]
'''

# Merge Garmin data with wearing off periods
#   Then, update columns
# ガーミンデータを着用オフ期間とマージします
#   それから、列を更新します
combined_data = pysqldf(cond_join).rename(columns=WEARING_OFF_COLUMNS)

# Drop duplicates based on timestamp
# タイムスタンプを基準に重複を削除します
combined_data.drop_duplicates(subset='timestamp', inplace=True)

# Set timestamp as index, of type DateTimeIndex
# タイムスタンプをインデックスとして設定します。DateTimeIndexのタイプ
combined_data.set_index('timestamp', inplace=True)
combined_data.index = pd.to_datetime(combined_data.index)

In [76]:
# Test set
# テストセット

# Since there is no available wearing-off periods,
#   copy Garmin data and apply same process as above
# 利用可能な着用オフ期間がないため、
#   ガーミンデータをコピーして、上記と同じプロセスを適用します
combined_data_test = garmin_data_test.copy().reset_index() \
  .rename(columns=WEARING_OFF_COLUMNS)

# Drop duplicates based on timestamp
# タイムスタンプを基準に重複を削除します
combined_data_test.drop_duplicates(subset='timestamp', inplace=True)

# Set timestamp as index, of type DateTimeIndex
# タイムスタンプをインデックスとして設定し、DateTimeIndexのタイプにします
combined_data_test.set_index('timestamp', inplace=True)
combined_data_test.index = pd.to_datetime(combined_data_test.index)

In [77]:
# Train set
# 訓練セット


# Add TARGET_FREQ to the start date
#   to get the start and end date of the wearing off period
#   e.g., 15min -> 15min after
# 開始日にTARGET_FREQを追加します
#   着用オフ期間の開始日と終了日を取得するには
#   例：15分->後の15分
drug_intake_with_symptoms['started_at_after_buffer'] = (
  drug_intake_with_symptoms['started_at'] +
    np.timedelta64(TARGET_FREQ_AS_INT, TARGET_FREQ_UNIT2)
)


def pysqldf(q): return sqldf(q, globals())


cond_join = '''
  select 
    combined_data.*,
    drug_intake_with_symptoms.*,
    case
      when drug_intake_with_symptoms.[started_at] is not null THEN 1
    else 0
    end as 'drug_intake'
  from combined_data
  left join drug_intake_with_symptoms
  on combined_data.[timestamp] BETWEEN
  drug_intake_with_symptoms.[started_at] AND
    drug_intake_with_symptoms.[started_at_after_buffer]
'''

# Merge combined data with drug intake times
#   Then, update columns
# 薬物摂取時間と組み合わせたデータをマージします
#   それから、列を更新します
combined_data = pysqldf(cond_join).rename(columns=DRUG_INTAKE_COLUMNS)

# Drop duplicates based on timestamp
# タイムスタンプを基準に重複を削除します
combined_data.drop_duplicates(subset='timestamp', inplace=True)
combined_data.drop(
  columns=['started_at_after_buffer'],
  inplace=True)

# Set timestamp as index, of type DateTimeIndex
# タイムスタンプをインデックスとして設定します。DateTimeIndexのタイプ
combined_data.set_index('timestamp', inplace=True)
combined_data.index = pd.to_datetime(combined_data.index)

In [78]:
wo_symptoms = ['wo_pain', 'wo_tremors', 'wo_anxiety', 'wo_rigidity',
               'wo_slowdown', 'wo_slow_thoughts', 'wo_impairment_hands',
               'wo_moodchange', 'wo_muscle_spasm']

drug_intake_symptoms = ['drug_intake_pain', 'drug_intake_tremors',
                        'drug_intake_anxiety', 'drug_intake_rigidity',
                        'drug_intake_slowdown', 'drug_intake_slow_thoughts',
                        'drug_intake_impairment_hands',
                        'drug_intake_moodchange', 'drug_intake_muscle_spasm']

symptoms = ['pain', 'tremors', 'anxiety', 'rigidity', 'slowdown', 'slow_thoughts',
            'impairment_hands', 'moodchange', 'muscle_spasm',
            'wearing_off_post_meds']


def generate_final_symptoms(row):
  """
  Generate a list of final symptoms based on the patient's symptoms.
  患者の症状に基づいて、最終的な症状のリストを生成します。
  """
  values = []
  for wo_symptom, drug_intake_symptom in zip(wo_symptoms, drug_intake_symptoms):
    # If the patient did not take medication, use the "without medication" symptom.
    # 患者が薬を服用しなかった場合は、「薬なし」の症状を使用します。
    if math.isnan(row[drug_intake_symptom]):
      if row[wo_symptom] is None or math.isnan(row[wo_symptom]):
        values.append(0)
      else:
        values.append(row[wo_symptom])
    # Otherwise, use the "after medication" symptom.
    # それ以外の場合は、「薬後」の症状を使用します。
    else:
      values.append(row[drug_intake_symptom])
  # Add the "wearing off" symptom if any of the symptoms are present.
  # 症状がいずれかある場合は、「着用オフ」の症状を追加します。
  if sum(values) >= 1:
    values.append(1)
  else:
    values.append(0)
  return pd.Series(values)

In [79]:

# Train set
# 訓練セット
final_symptoms = combined_data.apply(
  lambda row: generate_final_symptoms(row), axis=1
)

final_symptoms.columns = symptoms
combined_data = combined_data.merge(final_symptoms, on='timestamp', how='left')

In [80]:
def combine_wearing_offs(n):
  '''
  Check if the sum of the symptoms is greater than 0.
  If so, return 1, else return 0.
  That means there is at least 1 symptom present.
  症状の合計が0より大きいかどうかを確認します。
  そうであれば、1を返します。そうでなければ0を返します。
  つまり、少なくとも1つの症状が存在することを意味します。
  '''
  if n > 0:
    return 1
  else:
    return 0

In [81]:
# TRAIN SET
combined_data["final_wearing_off"] = (
  combined_data.wearing_off + combined_data.wearing_off_post_meds
).apply(
  lambda n: combine_wearing_offs(n)
).values

In [82]:
display(
  combined_data.head(5)
)

display(
  combined_data.columns
)

,heart_rate,steps,stress_score,awake,light,rem,deep,nonrem_total,total,nonrem_percentage,...,tremors,anxiety,rigidity,slowdown,slow_thoughts,impairment_hands,moodchange,muscle_spasm,wearing_off_post_meds,final_wearing_off
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-11-25 00:00:00,NaN,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:15:00,NaN,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:30:00,43.419355,0.0,-0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:45:00,60.250000,0.0,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 01:00:00,10.600000,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Index(['heart_rate', 'steps', 'stress_score', 'awake', 'light', 'rem', 'deep',
       'nonrem_total', 'total', 'nonrem_percentage', 'sleep_efficiency',
       'wearing_off_id', 'Unnamed: 0', 'wo_start', 'wo_end', 'wo_pain',
       'wo_tremors', 'wo_anxiety', 'wo_rigidity', 'wo_slowdown',
       'wo_slow_thoughts', 'wo_impairment_hands', 'wo_moodchange',
       'wo_muscle_spasm', 'wearing_off', 'drug_intake_id', 'Unnamed: 0',
       'drug_intake_start', 'drug_intake_end', 'drug_intake_pain',
       'drug_intake_tremors', 'drug_intake_anxiety', 'drug_intake_rigidity',
       'drug_intake_slowdown', 'drug_intake_slow_thoughts',
       'drug_intake_impairment_hands', 'drug_intake_moodchange',
       'drug_intake_muscle_spasm', 'drug_intake', 'pain', 'tremors', 'anxiety',
       'rigidity', 'slowdown', 'slow_thoughts', 'impairment_hands',
       'moodchange', 'muscle_spasm', 'wearing_off_post_meds',
       'final_wearing_off'],
      dtype='object')

In [83]:
# TRAIN SET
# 訓練セット
combined_data['wo_duration'] = (
  pd.to_datetime(combined_data.index) -
  pd.to_datetime(combined_data['wo_start'])
) / np.timedelta64(1, TARGET_FREQ_UNIT2)
combined_data['wo_duration'] = combined_data['wo_duration'].fillna(0)

combined_data.loc[:, ['final_wearing_off',
                      'wo_duration', 'wo_start']].iloc[123:139]

,final_wearing_off,wo_duration,wo_start
timestamp,,,
2021-11-26 06:45:00,0,0.0,None
2021-11-26 07:00:00,0,0.0,None
2021-11-26 07:15:00,1,5.0,2021-11-26 07:10:00.000000
2021-11-26 07:30:00,1,20.0,2021-11-26 07:10:00.000000
2021-11-26 07:45:00,1,35.0,2021-11-26 07:10:00.000000
2021-11-26 08:00:00,1,50.0,2021-11-26 07:10:00.000000
2021-11-26 08:15:00,0,0.0,None
2021-11-26 08:30:00,0,0.0,None
2021-11-26 08:45:00,0,0.0,None


In [84]:
# TRAIN SET
# 訓練セット
combined_data['time_from_last_drug_taken'] = (
  pd.to_datetime(combined_data.index) -
  pd.to_datetime(combined_data['drug_intake_start'])
) / np.timedelta64(1, TARGET_FREQ_UNIT2)

# Then, fill records after drug intake report by adding target frequency
# 次に、ターゲット周波数を追加して、薬の服用報告後のレコードを埋めます
gid = combined_data['time_from_last_drug_taken'].notnull().cumsum()
dg = combined_data.groupby(gid)
base = dg['time_from_last_drug_taken'].transform('last')
combined_data['time_from_last_drug_taken'] = (
  base + (dg.cumcount()) * TARGET_FREQ_AS_INT)

if combined_data['time_from_last_drug_taken'].isna().any():
  combined_data['time_from_last_drug_taken'] = \
    combined_data['time_from_last_drug_taken'].fillna(0)

combined_data.loc[:, ['drug_intake_id', 'drug_intake_start',
                      'time_from_last_drug_taken']].iloc[40:70]

,drug_intake_id,drug_intake_start,time_from_last_drug_taken
timestamp,,,
2021-11-25 10:00:00,1023.0,2021-11-25 09:54:00.000000,6.0
2021-11-25 10:15:00,NaN,None,21.0
2021-11-25 10:30:00,NaN,None,36.0
2021-11-25 10:45:00,NaN,None,51.0
2021-11-25 11:00:00,1024.0,2021-11-25 11:00:00.000000,0.0
2021-11-25 11:15:00,1024.0,2021-11-25 11:00:00.000000,15.0
2021-11-25 11:30:00,NaN,None,30.0
2021-11-25 11:45:00,NaN,None,45.0
2021-11-25 12:00:00,NaN,None,60.0


In [85]:
# TRAIN SET
# 訓練セット
combined_data['timestamp_hour'] = combined_data.index.hour
combined_data['timestamp_dayofweek'] = combined_data.index.dayofweek

In [86]:
# TEST SET
# テストセット
combined_data_test['timestamp_hour'] = combined_data_test.index.hour
combined_data_test['timestamp_dayofweek'] = combined_data_test.index.dayofweek

In [87]:
# TRAIN SET
# 訓練セット

# Fix timestamp format
# タイムスタンプ形式を修正する
date_time = pd.to_datetime(combined_data.index, format='%d.%m.%Y %H:%M:%S')

# Convert to timestamp
# タイムスタンプに変換する
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# Get seconds per day
# 1日あたりの秒数を取得する
day = 24 * 60 * 60
# Get seconds per year
# 1年あたりの秒数を取得する
year = 365.2425 * day

# Get sine(), cosine() for hour-feature
# hour-featureのsine()、cosine()を取得する
combined_data['timestamp_hour_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
combined_data['timestamp_hour_cos'] = np.cos(timestamp_s * (2 * np.pi / day))

In [88]:
# TEST SET
# テストセット

# Fix timestamp format
# タイムスタンプ形式を修正する
date_time = pd.to_datetime(combined_data_test.index, format='%d.%m.%Y %H:%M:%S')

# Convert to timestamp
# タイムスタンプに変換する
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# Get seconds per day
# 1日あたりの秒数を取得する
day = 24 * 60 * 60
# Get seconds per year
# 1年あたりの秒数を取得する
year = 365.2425 * day

# Get sine(), cosine() for hour-feature
# hour-featureのsine()、cosine()を取得する
combined_data_test['timestamp_hour_sin'] = np.sin(
    timestamp_s * (2 * np.pi / day))
combined_data_test['timestamp_hour_cos'] = np.cos(
    timestamp_s * (2 * np.pi / day))

In [89]:
combined_data_test.head(10)

,heart_rate,steps,stress_score,awake,light,rem,deep,nonrem_total,total,nonrem_percentage,sleep_efficiency,timestamp_hour,timestamp_dayofweek,timestamp_hour_sin,timestamp_hour_cos
timestamp,,,,,,,,,,,,,,,
2021-12-02 00:00:00,-1.000000,0.0,-1.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,0,3,-6.641221e-12,1.000000
2021-12-02 00:15:00,-1.000000,0.0,-1.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,0,3,6.540313e-02,0.997859
2021-12-02 00:30:00,-1.000000,0.0,-1.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,0,3,1.305262e-01,0.991445
2021-12-02 00:45:00,-1.000000,0.0,-1.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,0,3,1.950903e-01,0.980785
2021-12-02 01:00:00,-1.000000,-1.0,-1.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,1,3,2.588190e-01,0.965926
2021-12-02 01:15:00,19.133333,15.0,7.4,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,1,3,3.214395e-01,0.946930
2021-12-02 01:30:00,86.050000,0.0,48.4,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,1,3,3.826834e-01,0.923880
2021-12-02 01:45:00,88.366667,0.0,55.0,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,1,3,4.422887e-01,0.896873
2021-12-02 02:00:00,79.650000,-1.0,14.4,0.0,119.0,0.0,86.0,205.0,205.0,1.0,1.0,2,3,5.000000e-01,0.866025


In [90]:
# Choose features 特徴量
# Garmin features Garminの特徴量
features = ['heart_rate', 'steps', 'stress_score',
            'awake', 'deep', 'light', 'rem',
            'nonrem_total', 'total', 'nonrem_percentage', 'sleep_efficiency']

# Additional features
# 追加の特徴量
features += ['timestamp_hour', 'timestamp_dayofweek',
             'timestamp_hour_sin', 'timestamp_hour_cos']

# Choices: wearing_off | wearing_off_post_meds | final_wearing_off
# 選択肢：wearing_off | wearing_off_post_meds | final_wearing_off
TARGET_COLUMN = 'final_wearing_off'
features.append(TARGET_COLUMN)

In [94]:
# TRAIN SET
# 訓練セット

# Select features to include
# 含める特徴量を選択する
for_saving = combined_data.loc[:, features].copy()

# Rename target column to 'wearing_off
# ターゲット列を「wearing_off」に名前を変更する
for_saving.rename(columns={
  TARGET_COLUMN: 'wearing_off'
}, inplace=True)

# Add back the participant column
# 参加者の列を追加する
for_saving['participant'] = USER

# Save combined dataset to file
#  If the Excel file exists, append to the existing file
#  If the Excel file does not exist, create a new file
# Excelファイルが存在する場合は、既存のファイルに追加します
#  Excelファイルが存在する場合は、既存のファイルに追記する
#  Excelファイルが存在しない場合は、新規にファイルを作成します。
for_saving_file = f'D:\\UEL\\Disertation\\Parkinsons Disease\\Train Dataset\\Jupyter train\\combined_data.xlsx'
if os.path.exists(for_saving_file):
  with pd.ExcelWriter(for_saving_file, mode='a', if_sheet_exists='overlay') as writer:
    for_saving.reset_index().to_excel(
      writer, sheet_name='combined_dataset',
      startrow=writer.sheets['combined_dataset'].max_row,
      header=None, index=False
    )
else:
  with pd.ExcelWriter(for_saving_file) as writer:
    for_saving.to_excel(writer, sheet_name='combined_dataset')

In [93]:

# TEST SET
# テストセット

# Select features to include
# 含める特徴量を選択する
combined_data_test[TARGET_COLUMN] = np.nan
combined_data_test = combined_data_test.reset_index().rename(
  columns={"Timestamp": "timestamp"}).set_index('timestamp')
for_saving = combined_data_test.loc[:, features].copy()

# Rename target column to 'wearing_off
# ターゲット列を「wearing_off」に名前を変更する
for_saving.rename(columns={
  TARGET_COLUMN: 'wearing_off'
}, inplace=True)

# Add back the participant column
# 参加者の列を追加する
for_saving['participant'] = USER

# Save combined dataset to file
#  If the Excel file exists, append to the existing file
#  If the Excel file does not exist, create a new file
# Excelファイルが存在する場合は、既存のファイルに追加します
#  Excelファイルが存在する場合は、既存のファイルに追記する
#  Excelファイルが存在しない場合は、新規にファイルを作成します。
for_saving_file = f'D:\\UEL\\Disertation\\Parkinsons Disease\\Test Dataset\\Jupyter\\combined_data.xlsx'
if os.path.exists(for_saving_file):
  with pd.ExcelWriter(for_saving_file, mode='a', if_sheet_exists='overlay') as writer:
    for_saving.reset_index().to_excel(
      writer, sheet_name='combined_dataset',
      startrow=writer.sheets['combined_dataset'].max_row,
      header=None, index=False
    )
else:
  with pd.ExcelWriter(for_saving_file) as writer:
    for_saving.to_excel(writer, sheet_name='combined_dataset')